In [1]:
import ee
import geepy
import math

import branca.colormap as cm
import random

ee.Initialize()

In [2]:
config = geepy.params.configParams('input.json')

ls = ee.FeatureCollection(config.params['lsTiles'])
tiles = [geepy.feature.getTiles(ls, tile, 'TILE_T') for tile in config.params['grid']['tiles2process']]

l5 = ee.ImageCollection(config.params['imgCollection']['lc5']['id']).select(
    config.params["imgCollection"]["lc5"]["bands"],
    config.params["imgCollection"]["lc5"]["bandNames"])

l7 = ee.ImageCollection(config.params['imgCollection']['lc7']['id']).select(
    config.params["imgCollection"]["lc7"]["bands"],
    config.params["imgCollection"]["lc7"]["bandNames"])

l8 = ee.ImageCollection(config.params['imgCollection']['lc8']['id']).select(
    config.params["imgCollection"]["lc8"]["bands"],
    config.params["imgCollection"]["lc8"]["bandNames"])

# MapBiomas Coleção 2.3
mapBiomas = ee.Image(config.params['mapBiomas'])

# Modelo digital de elevação
srtm = ee.Image(config.params['srtm'])

# Localities distace
villages = ee.FeatureCollection(config.params['villages']);
towns = ee.FeatureCollection(config.params['towns']);
farms = ee.FeatureCollection(config.params['farms']);
hamlets = ee.FeatureCollection(config.params['hamlets']);
rivers = ee.FeatureCollection(config.params['rivers']);

dvillage = villages.distance(config.params['radist'])
dfarm = farms.distance(config.params['radist'])
dtown = towns.distance(config.params['radist'])
dhamlet = hamlets.distance(config.params['radist'])
driver = rivers.distance(config.params['radist'])

KeyError: 'mapBiomas'

In [ ]:
tileNames = [tiles[i][0] for i in range(len(tiles))]
tileCoords = [tiles[i][1].geometry().getInfo()['coordinates'] for i in range(len(tiles))]

In [ ]:
tileCoords

In [ ]:
slope = ee.Terrain.slope(srtm)
aspect = ee.Terrain.aspect(srtm).divide(180).multiply(math.pi).sin()

In [ ]:
for year in range(2016, 2017):
    y = str(year)
    start_d = config.params['years'][y]+config.params['period']['dry']['start']
    end_d = config.params['years'][y]+config.params['period']['dry']['end']
    #print("Start-End dry season %s %s" %(start_d, end_d))
    
    if(year < 2002):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(year in (2002,2011,2012)):
        filtered = l7.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    elif(year > 2002 and year < 2011):
        filtered = l5.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)
    else:
        filtered = l8.filterMetadata('CLOUD_COVER', 'less_than', config.params['cloudCoverThreshold']).filterDate(start_d, end_d)

    filt = filtered.median()
    
    #Landsat Tiles
    lsClip = [geepy.image.clipImageCollection(filt, tiles[i][1]) for i in range(len(tiles))]
    
    #VIIRS Tiles
    viirs = ee.Image(config.params['VIIRS'][str(year)]).select('avg_rad').divide(100)
    
    #Resample VIIRS to 30 m
    ntl_30m = viirs.resample('bilinear').reproject(
        crs = viirs.projection().crs(),
        scale = 30
    )
    
    viirsClip = [geepy.image.clipImageCollection(ntl_30m, tiles[i][1]) for i in range(len(tiles))]
    srtmClip = [geepy.image.clipImageCollection(srtm, tiles[i][1]) for i in range(len(tiles))]
    slopeClip = [geepy.image.clipImageCollection(slope, tiles[i][1]) for i in range(len(tiles))]
    aspectClip = [geepy.image.clipImageCollection(aspect, tiles[i][1]) for i in range(len(tiles))]
    
    dvillageClip = [geepy.image.clipImageCollection(dvillage, tiles[i][1]) for i in range(len(tiles))]
    dfarmClip = [geepy.image.clipImageCollection(dfarm, tiles[i][1]) for i in range(len(tiles))]
    dtownClip = [geepy.image.clipImageCollection(dtown, tiles[i][1]) for i in range(len(tiles))]
    dhamletClip = [geepy.image.clipImageCollection(dhamlet, tiles[i][1]) for i in range(len(tiles))]
    driverClip = [geepy.image.clipImageCollection(driver, tiles[i][1]) for i in range(len(tiles))]

In [ ]:
for i in range(len(lsClip)):
    lsClip[i] = geepy.image.calcNDVI(lsClip[i])
    lsClip[i] = geepy.image.calcEVI(lsClip[i])
    lsClip[i] = geepy.image.calcSAVI(lsClip[i])
    lsClip[i] = geepy.image.calcNDWI(lsClip[i])
    lsClip[i] = geepy.image.calcMNDWI(lsClip[i])
    lsClip[i] = geepy.image.waterMask(lsClip[i])
    
    lsClip[i] = geepy.image.calcFractions(lsClip[i])
    lsClip[i] = geepy.image.calcNDFI(lsClip[i])
    lsClip[i] = geepy.image.calcNDFI2(lsClip[i])
    lsClip[i] = geepy.image.calcNDFI3(lsClip[i])
    lsClip[i] = geepy.image.calcFCI(lsClip[i])
    lsClip[i] = geepy.image.calcGVNPV(lsClip[i])
    lsClip[i] = geepy.image.calcNPVSOIL(lsClip[i])
    lsClip[i] = geepy.image.img2Band(lsClip[i], viirsClip[i], 'ntl')
    lsClip[i] = geepy.image.calcNDBI(lsClip[i])
    

In [ ]:
for i in range(2016, 2017):
    dt = 'classification_'+ str(i)  
    mbClip = [geepy.image.clipImageCollection(mapBiomas.select(dt), tiles[i][1]) for i in range(len(tiles))]

In [ ]:
for i in range(len(mbClip)):
    mbClip[i] = mbClip[i].addBands(ee.Image.pixelLonLat())
    mbClip[i] = mbClip[i].rename(["class", "longitude", "latitude"]).cast({"class": "int8"})
    lsClip[i] = geepy.image.img2Band(lsClip[i], srtmClip[i], 'srtm')
    lsClip[i] = geepy.image.img2Band(lsClip[i], slopeClip[i], 'slope')
    lsClip[i] = geepy.image.img2Band(lsClip[i], aspectClip[i], 'aspect')
    
    lsClip[i] = geepy.image.img2Band(lsClip[i], dvillageClip[i], 'dvillage')
    lsClip[i] = geepy.image.img2Band(lsClip[i], dfarmClip[i], 'dfarm')
    lsClip[i] = geepy.image.img2Band(lsClip[i], dtownClip[i], 'dtown')
    lsClip[i] = geepy.image.img2Band(lsClip[i], dhamletClip[i], 'dhamlet')
    lsClip[i] = geepy.image.img2Band(lsClip[i], driverClip[i], 'driver')
    lsClip[i] = geepy.image.img2Band(lsClip[i], mbClip[i].select('class'), 'mbiomas')

In [ ]:
# for i in range(len(lsClip)):
#     lsClip[i] = geepy.image.img2Band(lsClip[i], mbClip[i], 'mpmask')

In [ ]:
# Projeto AIBA

# 1  Floresta
# 2  Formações Florestais
# 3  Formações savanicas
# 4  Mangue
# 5  Florestas plantadas
# 9  Formações naturais não-florestais
# 10  Áreas umidas naturais não fllorestais
# 11  Campos
# 12  Outras formações não florestais
# 13 Uso agropecuário
# 14 Pastagem
# 15 Agricultura
# 18 Agricultura ou pastagem
# 21 Areas não vegetadas
# 22 Praias e dunas
# 23 Urbano
# 24 Outras áreas não vegetadas
# 25 agua
# 26 não-observado


# stratifiedSample

n = 1000;

tileIndex = int(tileNames.index('T220068'))

mb_classes = [i for i in range(28)]

# classes específicas
points = mbClip[tileIndex].stratifiedSample(
    numPoints = n,
    classBand = "class", 
    region = tiles[tileIndex][1], 
    seed = 2015, #random.randint(1,1000),
    #1,2,3,4,5,9,10,11,12,13,14,15,18,21,22,23,24,25,26,27
    classValues = mb_classes, # valores a serem classificados 
    classPoints = [0,0,0,n,n,
                   n,n,n,0,n,
                   0,n,n,n,0,
                   n,n,n,n,n,
                   0,n,n,n,n,
                   n,n,n],
    dropNulls = True, 
    scale = 30
)   

In [ ]:
bands = config.params['bandParams']

In [ ]:
points = points.map(geepy.feature.setGeometry)

In [ ]:
# Training
training = lsClip[tileIndex].sampleRegions(
    collection = points,
    properties = ['class'],
    scale = 30,
) 

In [ ]:
trained = ee.Classifier.randomForest(10).train(training, 'class', bands)
classified_rf = lsClip[tileIndex].select(bands).classify(trained)

In [ ]:
#final = classified_rf.toInt16()
final = classified_rf.toByte()

#Ultima versão
remapped = final.remap([3,4,5,6,7,9,11,12,13,15,16,17,18,19,21,22,23,24,25,26,27], \
                      [1,2,2,3,3,3,2, 2, 2, 4, 2, 2, 3, 2, 3, 3, 2, 5, 5, 6, 2])


#remaped = final.remap([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27], \
#                      [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27])

In [ ]:
#lsClip[tileIndex].bandNames().getInfo()

In [ ]:
mapa_c = geepy.maps.geeMap(tiles[tileIndex][1],zoom=10)

l8_vis_params = {'min':0,'max':0.3,'bands':'red,green,blue','format':'png'}

class_visParams = {'min':1, 'max':6, 'palette':'006400,00ff00,ffcfff,fbf3c7,ff0000,0000ff', 'format': 'png'}

mapBiomas_visParams = {'min': 0, 'max':28,'palette':'d5d5e5,129912,1f4423,006400,00ff00,687537,\
                    76a5af,29eee4,77a605,935132,ff9966,45c2a5, \
                    b8af4f,f1c232,ffffb2,ffd966,f6b26b,a0d0de, \
                    e974ed,d5a6bd,c27ba0,FBF3C7,ea9999,dd7e6b, \
                    b7b7b7,ff99ff, 0000ff,d5d5e5,ce3d3d', 'format': 'png'}

# mapa_c.addLayer(lsClip[tileIndex].select('ndwi'), name='ndwi')
# mapa_c.addLayer(lsClip[tileIndex].select('ndfi'), vis_params = {'min': 0, 'max':200}, name='ndfi')

mapa_c.addLayer(lsClip[tileIndex], vis_params=l8_vis_params, name='landsat')
#mapa_c.addLayer(mbClip[tileIndex].select('class'), vis_params=mapBiomas_visParams, name='MapBiomas')
mapa_c.addLayer(remapped,vis_params=class_visParams, name='classified_rf')

#mapa_c.addLayer(lsClip[tileIndex].select(['wmask']), vis_params = mapBiomas_visParams, name='wmask')
#mapa_c.addLayer(lsClip[tileIndex].select(['mndwi']), vis_params = {'min': 0,'max': 1}, name='mndwi')

#mapa_c.addLayer(lsClip[tileIndex].select(['srtm']), vis_params = {'min':0,'max':1000, 'palette':'76a5af,29eee4,77a605,935132,ff9966,45c2a5'}, name='srtm')

#mapa_c.addLayer(lsClip[tileIndex].select('ntl'), name='viirs')

# mapa_c.addLayer(lsClip[tileIndex].select('gv'), vis_params = {'min': 0, 'max':0.004}, name='gv')
# mapa_c.addLayer(lsClip[tileIndex].select('npv'), vis_params = {'min': 0, 'max':0.002}, name='npv')
# mapa_c.addLayer(lsClip[tileIndex].select('soil'), vis_params = {'min': 0., 'max':0.004}, name='soil')
# mapa_c.addLayer(lsClip[tileIndex].select('gvs'), name='gvs')
# mapa_c.addLayer(lsClip[tileIndex], vis_params = {'min': 0, 'max':0.004, 'bands': 'gv, npv, soil'}, name='frac')

mapa_c.addLayer(points,vis_params = {'color': 'ff0000'}, name='pontos')

mapa_c.addControls()   

In [ ]:
#task = geepy.image.send2asset(remapped, tileCoords[tileIndex], 'Classification2', 'users/fernandompimenta/AIBA/classification2')

In [ ]:
#task.start()

In [ ]:
#Export the FeatureCollection to a KML file.
task = ee.batch.Export.table.toDrive(
  collection = points,
  description ='amostras',
  fileFormat = 'GeoJSON'
)

In [ ]:
task.start()

In [ ]:
                            # '006400,00ff00,ffcfff,fbf3c7,ff0000,d5d5e5,0000ff'
colormap = cm.LinearColormap(['#006400','#00ff00','#ffcfff','#fbf3c7','#ff0000','#000000','#0000ff'],
    vmin=1,
    vmax=7
).to_step(7)


colormapbiomas = cm.LinearColormap(['#d5d5e5','#129912','#1f4423','#006400','#00ff00','#687537',\
                    '#76a5af','#29eee4','#77a605','#935132','#ff9966','#45c2a5', \
                    '#b8af4f','#f1c232','#ffffb2','#ffd966','#f6b26b','#a0d0de', \
                    '#e974ed','#d5a6bd','#c27ba0','#FBF3C7','#ea9999','#dd7e6b', \
                    '#b7b7b7','#ff99ff','#0000ff','#d5d5e5','#ce3d3d',],
    vmin=0,
    vmax=28
).to_step(29)

colormapbiomas

In [ ]:
colormap

In [ ]:
task = geepy.image.send2drive(mbClip[tileIndex].select('class'), tileCoords[tileIndex], 'mapbiomasClass', 'TESTEEE')

In [ ]:
task.start()

In [ ]:
task = geepy.image.send2drive(classified_rf, tileCoords[tileIndex], 'class_new_27', 'driveDir')

In [ ]:
task.start()